In [1]:
%matplotlib inline

In [2]:
import os
import pyxdf
import numpy as np
import mne
import matplotlib.pyplot as plt
import scipy

### Settings

In [3]:
# Files
subject = 'P00J'
session = 1
task = 'MI-hands'
lslDir = os.path.join(os.path.expanduser('~'), 'Documents\CurrentStudy')

# LSL Stream
eeg_stream_type = 'EXG'
markers_stream_type = 'Marker'

# Experiment parameters
rest_duration = 5
task_duration = 5

# Events
event_dict = {'rest': 0, 'MI/hands': 1}
tmin = 0
tmax = 5

In [4]:
# Find files
xdf_files = []
hasSubject = subject!=''
hasSession = session!=''
hasTask = task!=''
for root, dir, files in os.walk(lslDir):
    validFile = True
    for file in files:
        if hasSubject:
            validFile = validFile and (subject in file)
        if hasSession:
            validFile = validFile and (('ses-S' + str(session).zfill(3)) in file);
        if hasTask:
            validFile = validFile and (task in file)
        if validFile:
            matchingFile = os.path.join(root, file)
            print(matchingFile)
            xdf_files.append(matchingFile)

C:\Users\leungj\Documents\CurrentStudy\sub-P00J\ses-S001\eeg\sub-P00J_ses-S001_task-MI-hands_run-001_eeg.xdf
C:\Users\leungj\Documents\CurrentStudy\sub-P00J\ses-S001\eeg\sub-P00J_ses-S001_task-MI-hands_run-002_eeg.xdf
C:\Users\leungj\Documents\CurrentStudy\sub-P00J\ses-S001\eeg\sub-P00J_ses-S001_task-MI-hands_run-003_eeg.xdf
C:\Users\leungj\Documents\CurrentStudy\sub-P00J\ses-S001\eeg\sub-P00J_ses-S001_task-MI-hands_run-004_eeg.xdf
C:\Users\leungj\Documents\CurrentStudy\sub-P00J\ses-S001\eeg\sub-P00J_ses-S001_task-MI-hands_run-005_eeg.xdf


In [ ]:
# Parse streams
eeg_stream, marker_stream = [], []

print('Parsing streams')
for xdf_file in xdf_files:
    streams, header = pyxdf.load_xdf(xdf_file)
    for i in range(len(streams)):
        if streams[i]['info']['type'][0] == eeg_stream_type:
            print("Found %s stream in %s" % (eeg_stream_type, os.path.basename(xdf_file)))
            eeg_stream.append(streams[i])
        elif streams[i]['info']['type'][0] == markers_stream_type:
            print("Found %s stream in %s" % (markers_stream_type, os.path.basename(xdf_file)))
            marker_stream.append(streams[i])
del streams, header

In [ ]:
# Extract EEG data
print("Extracting EEG info")

ch_names = []
if eeg_stream[0]['info']['desc'][0]:
    print("EEG channel names found")
    for i in range(len(eeg_stream[0]['info']['desc'][0]['channels'][0]['channel'])):
        ch_names.append(eeg_stream[0]['info']['desc'][0]['channels'][0]['channel'][i]['label'][0])
else:
    print("EEG channel names not found... setting default")
    ch_names = ['FP1', 'FP2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T7', 'T8', 'P3', 'P4']
print('Channels: ', ch_names)

sfreq = float(eeg_stream[0]['info']['nominal_srate'][0])
print('Sampling frequency: ', sfreq)

# Create MNE info object
eeg_info = mne.create_info(ch_names, sfreq, ch_types='eeg')

In [ ]:
montage = mne.channels.read_custom_montage('openbci_montage.elc')

In [ ]:
# Get all EEG data
eeg_epoch_list = []

for n in range(len(eeg_stream)):
    # Create MNE Raw object
    eeg_data = np.transpose(eeg_stream[n]['time_series'])
    eeg_data = eeg_data / 1e6
    print(eeg_data.shape)
    eeg_raw = mne.io.RawArray(eeg_data, eeg_info)
    
    # Set montage
    eeg_raw.set_montage(montage)

    # Add annotations
    onset, duration, description = [], [], []
    for i in range(len(marker_stream[n]['time_series'])):
        if ('rest' in marker_stream[n]['time_series'][i][0]):
            onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
            duration.append(rest_duration)
            description.append(marker_stream[n]['time_series'][i][0])
        elif ('task' in marker_stream[n]['time_series'][i][0]):
            onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
            duration.append(task_duration)
            description.append(marker_stream[n]['time_series'][i][0].replace('task_', '').replace('-','/'))
    annotations = mne.Annotations(onset, duration, description)
    eeg_raw = eeg_raw.set_annotations(annotations)
    
    # Epoch data
    events, event_id = mne.events_from_annotations(eeg_raw, event_id=event_dict)
    eeg_epoch = mne.Epochs(eeg_raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, picks='eeg', preload=True, detrend=1)
    eeg_epoch_list.append(eeg_epoch)
    print(eeg_epoch)

In [ ]:
# concatenate all epochs
eeg_epochs = mne.concatenate_epochs(eeg_epoch_list)
eeg_epochs

In [ ]:
# Re-reference data
eeg_epochs = eeg_epochs.set_eeg_reference ('average', projection=False)

In [ ]:
# Filter data
l_freq = 40.
h_freq = 1.
eeg_epochs = eeg_epochs.filter(l_freq, h_freq, n_jobs=-1)

In [ ]:
# plot eeg data
fig = eeg_epochs.plot(title='Raw Data', n_channels=len(ch_names), show=True)

In [ ]:
# print(eeg_epochs)

fig = eeg_epochs['rest'].plot_psd()
fig = eeg_epochs['MI/hands'].plot_psd()

In [ ]:
rest_data = eeg_epochs['rest'].get_data()
MI_data = eeg_epochs['MI/hands'].get_data()
f, Pxx_den_rest = scipy.signal.welch(np.mean(rest_data, 0), fs=125.)
f, Pxx_den_MI = scipy.signal.welch(np.mean(MI_data, 0), fs=125.)


channel = 3
plt.title('Welch PSD for channel: %s' % ch_names[channel])
plt.semilogy(f, Pxx_den_rest[channel,:])
plt.semilogy(f, Pxx_den_MI[channel,:])
plt.xlim([0, 30])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.legend(['rest', 'MI/hands'])
plt.show()